In [1]:
import jupyter_black
import black
import pandas as pd
import gc
from handlers import *

gc.collect()

jupyter_black.load(
    lab=False,
    line_length=100,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

DEBUG:jupyter_black:config: {'line_length': 100, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

In [2]:
# Split the v_extract1.pdt_capacity table

split_bigquery_table_into_chunks(dataset_id="v_extract1", table_id="pdt_capacity", chunk_size=5000000)

100%|██████████| 25/25 [10:48<00:00, 25.93s/it]


In [2]:
chunk_names = [chunk for chunk in list_bigquery_tables("chunked") if chunk.startswith("pdt_capacity")]
df_test = read_bigquery_table(dataset_id="chunked", table_id=chunk_names[0], key_field="PRODUCTCODE")

In [3]:
df_test = df_test

,PRODUCTCODE,pdt_capacity_chunk_1_TOURGRADECODE,pdt_capacity_chunk_1_AVAILABILITYSTATUS,pdt_capacity_chunk_1_CAPACITY,pdt_capacity_chunk_1_CONSUMEDBYADULT,pdt_capacity_chunk_1_CONSUMEDBYSENIOR,pdt_capacity_chunk_1_CONSUMEDBYYOUTH,pdt_capacity_chunk_1_CONSUMEDBYCHILD,pdt_capacity_chunk_1_CONSUMEDBYINFANT,pdt_capacity_chunk_1_STARTTIME,pdt_capacity_chunk_1_AVAILABILITYDATE,pdt_capacity_chunk_1_BOOKINGCUTOFF,pdt_capacity_chunk_1_BOOKINGCUTOFFUTC,pdt_capacity_chunk_1_UPDATEDAT,pdt_capacity_chunk_1_UPDATEDATUTC
0,2218P9,TG1,AVAILABLE,25,True,True,True,True,True,15:53:10.144000,2023-09-30,2023-09-28 18:00:00+00:00,2023-09-28 18:00:00+00:00,2023-09-18 21:21:18.419471+00:00,2023-09-18 21:21:18.419471+00:00
1,2218P9,TG1,AVAILABLE,25,True,True,True,True,True,15:53:10.144000,2023-09-24,2023-09-22 18:00:00+00:00,2023-09-22 18:00:00+00:00,2023-09-21 04:49:02.555286+00:00,2023-09-21 04:49:02.555286+00:00
2,2625MC,BME,AVAILABLE,39,True,False,False,True,False,11:23:10.144000,2023-11-21,2023-11-21 04:59:00+00:00,2023-11-21 04:59:00+00:00,2023-11-14 17:02:30.916646+00:00,2023-11-14 17:02:30.916646+00:00
3,2625MC,BME,AVAILABLE,39,True,False,False,True,False,NaT,2023-12-20,2023-12-20 04:59:00+00:00,2023-12-20 04:59:00+00:00,2023-12-13 15:55:19.954964+00:00,2023-12-13 15:55:19.954964+00:00
4,2625MC,BME,AVAILABLE,47,True,False,False,True,False,NaT,2024-01-22,2024-01-22 04:59:00+00:00,2024-01-22 04:59:00+00:00,2024-01-18 19:27:34.609662+00:00,2024-01-18 19:27:34.609662+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,5557BRKRENT,3HR,AVAILABLE,250,True,False,False,True,False,10:53:10.144000,2023-11-16,2023-11-17 16:00:00+00:00,2023-11-17 16:00:00+00:00,2023-11-10 17:18:12.576497+00:00,2023-11-10 17:18:12.576497+00:00
4999996,5557BRKRENT,3HR,AVAILABLE,250,True,False,False,True,False,13:53:10.144000,2023-09-28,2023-09-29 15:00:00+00:00,2023-09-29 15:00:00+00:00,2023-09-22 00:22:13.601292+00:00,2023-09-22 00:22:13.601292+00:00
4999997,5557BRKRENT,DAY,AVAILABLE,250,True,False,False,True,False,12:00:00,2023-04-15,2023-04-16 15:00:00+00:00,2023-04-16 15:00:00+00:00,2023-04-15 20:12:50.578339+00:00,2023-04-15 20:12:50.578000+00:00
4999998,5557BRKRENT,3HR,AVAILABLE,250,True,False,False,True,False,14:00:00,2023-07-31,2023-08-01 15:00:00+00:00,2023-08-01 15:00:00+00:00,2023-07-29 04:50:19.536322+00:00,2023-07-29 04:50:19.536000+00:00


In [ ]:
# Merge tags_cleaned and pdt_tourgrades

tags_cleaned = read_bigquery_table(dataset_id="v_extract1", table_id="tags_cleaned")
pdt_tourgrades = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_tourgrades")

product_step_1 = pd.merge(tags_cleaned, pdt_tourgrades, on='PRODUCTCODE', how='outer')
product_step_1 = product_step_1.astype(str)
product_step_1.replace("nan", "", inplace=True)

In [ ]:
del tags_cleaned
del pdt_tourgrades

In [ ]:
# Merge step result and pdt_tags

pdt_tags = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_tags")
pdt_tags = pdt_tags[["PRODUCTCODE", "pdt_tags_TAGLEVEL", "pdt_tags_TAGNAME"]]

product_step_2 = pd.merge(product_step_1, pdt_tags, on='PRODUCTCODE', how='outer')
product_step_2 = product_step_2.astype(str)
product_step_2.replace("nan", "", inplace=True)

In [ ]:
del product_step_1
del pdt_tags

In [ ]:
# Merge step result and pdt_product_level

pdt_product_level = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_product_level")

product_step_3 = pd.merge(product_step_2, pdt_product_level, on='PRODUCTCODE', how='outer')
product_step_3 = product_step_3.astype(str)
product_step_3.replace("nan", "", inplace=True)

In [ ]:
del product_step_2
del pdt_product_level

In [ ]:
# Merge step result and pdt_product_detail

pdt_product_detail = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_product_detail")

product_step_3.rename(columns={"pdt_tourgrades_TOURGRADECODE": 'TOURGRADECODE'}, inplace=True)
pdt_product_detail.rename(columns={"pdt_product_detail_TOURGRADECODE": 'TOURGRADECODE'}, inplace=True)

product_step_4 = pd.merge(product_step_3, pdt_product_detail, on=['PRODUCTCODE', 'TOURGRADECODE'], how='outer')
product_step_4 = product_step_4.astype(str)
product_step_4.replace("nan", "", inplace=True)

product_step_4 = aggregate(product_step_4, "PRODUCTCODE")

for col in tqdm(product_step_4.columns):

   if col != "PRODUCTCODE":

       product_step_4 = get_ordered_set_dict(product_step_4, col)

product_step_4.to_pickle("tmp1.pkl")

In [ ]:
del product_step_3
del pdt_product_detail

In [ ]:
product_step_4 = pd.read_pickle("tmp1.pkl")

In [ ]:
# Merge step result and pdt_itinerary

pdt_itinerary = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_itinerary")
pdt_itinerary = aggregate(pdt_itinerary, "PRODUCTCODE")

for col in tqdm(pdt_itinerary.columns):

   if col != "PRODUCTCODE":

       pdt_itinerary = get_ordered_set_dict(pdt_itinerary, col)

product_step_5 = pd.merge(product_step_4, pdt_itinerary, on='PRODUCTCODE', how='outer')
product_step_5 = product_step_5.astype(str)
product_step_5.replace("nan", "", inplace=True)

product_step_5.to_pickle("tmp2.pkl")

In [ ]:
del product_step_4
del pdt_itinerary

In [ ]:
product_step_5 = pd.read_pickle("tmp2.pkl")

In [ ]:
# Merge step result and pdt_inclexcl_ENG

pdt_inclexcl_ENG = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_inclexcl_ENG")
pdt_inclexcl_ENG = aggregate(pdt_inclexcl_ENG, "PRODUCTCODE")

for col in tqdm(pdt_inclexcl_ENG.columns):

   if col != "PRODUCTCODE":

       pdt_inclexcl_ENG = get_ordered_set_dict(pdt_inclexcl_ENG, col)

product_step_6 = pd.merge(product_step_5, pdt_inclexcl_ENG, on='PRODUCTCODE', how='outer')
product_step_6 = product_step_6.astype(str)
product_step_6.replace("nan", "", inplace=True)

product_step_6.to_pickle("tmp3.pkl")

In [ ]:
del product_step_5
del pdt_inclexcl_ENG

In [ ]:
product_step_6 = pd.read_pickle("tmp3.pkl")

In [ ]:
# Merge step result and pdt_inclexcl

pdt_inclexcl = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_inclexcl")
pdt_inclexcl = aggregate(pdt_inclexcl, "PRODUCTCODE")

for col in tqdm(pdt_inclexcl.columns):

   if col != "PRODUCTCODE":

       pdt_inclexcl = get_ordered_set_dict(pdt_inclexcl, col)

product_step_7 = pd.merge(product_step_6, pdt_inclexcl, on='PRODUCTCODE', how='outer')
product_step_7 = product_step_7.astype(str)
product_step_7.replace("nan", "", inplace=True)

product_step_7.to_pickle("tmp4.pkl")

In [ ]:
del product_step_6
del pdt_inclexcl

In [ ]:
product_step_7 = pd.read_pickle("tmp4.pkl")

In [ ]:
# Merge step result and pdt_incexcl

pdt_incexcl = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_incexcl")
pdt_incexcl = aggregate(pdt_incexcl, "PRODUCTCODE")

for col in tqdm(pdt_incexcl.columns):

   if col != "PRODUCTCODE":

       pdt_incexcl = get_ordered_set_dict(pdt_incexcl, col)

product_step_8 = pd.merge(product_step_7, pdt_incexcl, on='PRODUCTCODE', how='outer')
product_step_8 = product_step_8.astype(str)
product_step_8.replace("nan", "", inplace=True)

product_step_8.to_pickle("tmp5.pkl")

In [ ]:
del product_step_7
del pdt_incexcl

In [6]:
split_table_into_chunks(dataset_id="v_extract1", table_id="pdt_capacity", chunk_size=100000)